In [6]:
from crr_labels import fantom, roadmap, roadmap_available_cell_lines, fantom_available_cell_lines
import os
from tqdm.auto import tqdm
import pandas as pd
from tabulate import tabulate

In [7]:
common_cell_lines = [
    "GM12878", "HelaS3", "HepG2", "K562", "A549"
]
fantom_cell_lines = [
    "MCF7", "HEK293", "Caco2", "HL60", "PC3"
]
roadmap_cell_lines = [
    "H1", "DND41", "H9", "HUES48", "HUES6", "HUES64", "IMR90", "4STAR"
]

In [8]:
windows = (100, 200, 300, 500, 1000)
models = (15, 18)
centers = ("peak", "center")

In [9]:
url_pattern = "`Download <https://raw.githubusercontent.com/LucaCappelletti94/crr_labels/master/{url}>`__"

In [10]:
fantom_preprocessed = []
roadmap_preprocessed = []
for window in tqdm(windows):
    for center_enhancers in tqdm(centers, leave=False):
        path = f"preprocessed/fantom/window_size/{window}"
        enhancer_path = f"{path}/enhancers_{center_enhancers}.csv"
        promoter_path = f"{path}/promoters.csv"
        if os.path.exists(enhancer_path):
            continue
        enhancers, promoters = fantom(
            common_cell_lines+fantom_cell_lines, window, center_enhancers=center_enhancers)
        os.makedirs(path, exist_ok=True)
        enhancers.to_csv(enhancer_path, index=None)
        enhancers.to_csv(promoter_path, index=None)
    fantom_preprocessed.append({
        "Nucleotides window": window,
        "Genome":"hg19",
        "Region-centered enhancers": url_pattern.format(url=enhancer_path),
        "Peak-centered enhancers": url_pattern.format(url=enhancer_path),
        "Promoters": url_pattern.format(url=promoter_path)
    })
    for model in tqdm(models, leave=False):
        path = f"preprocessed/roadmap/model/{model}/window_size/{window}"
        enhancer_path = f"{path}/enhancers.csv"
        promoter_path = f"{path}/promoters.csv"
        roadmap_preprocessed.append({
            "Nucleotides window": window,
            "Genome":"hg19",
            "Enhancers": url_pattern.format(url=enhancer_path),
            "Promoters": url_pattern.format(url=promoter_path),
            "Model states": model
        })
        if os.path.exists(path):
            continue
        enhancers, promoters = roadmap(common_cell_lines+roadmap_cell_lines, window)
        os.makedirs(path, exist_ok=True)
        enhancers.to_csv(enhancer_path, index=None)
        enhancers.to_csv(promoter_path, index=None)

df = pd.DataFrame(roadmap_preprocessed)[[
    "Nucleotides window",
    "Genome",
    "Model states",
    "Enhancers",
    "Promoters"
]]
with open("roadmap.txt", "w") as f:
    f.write(tabulate(df.values, headers=df.columns, tablefmt="grid"))

df = pd.DataFrame(fantom_preprocessed)[[
    "Nucleotides window",
    "Genome",
    "Region-centered enhancers",
    "Peak-centered enhancers",
    "Promoters"
]]
with open("fantom.txt", "w") as f:
    f.write(tabulate(df.values, headers=df.columns, tablefmt="grid"))

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


ValueError: Given cell line IMR90 is not currently available.